In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from string import digits
import re
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Input, Dense,Embedding, Concatenate, TimeDistributed
from tensorflow.keras.models import Model,load_model, model_from_json
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import pickle as pkl
import numpy as np

In [95]:
with open('.//pol-eng//pol.txt','r',encoding="utf8") as f:
  data = f.read()

In [96]:
data[:6]

'Go.\tId'

# Data Preprocessing

In [97]:
uncleaned_data_list = data.split('\n')
len(uncleaned_data_list)
uncleaned_data_list[0]

'Go.\tIdź.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #7818294 (Hanyel)'

In [98]:
uncleaned_data_list = uncleaned_data_list[:40000]
len(uncleaned_data_list)

40000

In [99]:
english_word = []
pol_word = []
cleaned_data_list = []
for word in uncleaned_data_list:
  english_word.append(word.split('\t')[:-1][0])
  pol_word.append(word.split('\t')[:-1][1])

In [100]:
english_word

['Go.',
 'Hi.',
 'Run!',
 'Run.',
 'Run.',
 'Who?',
 'Wow!',
 'Wow!',
 'Fire!',
 'Fire!',
 'Fire!',
 'Help!',
 'Jump!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait.',
 'Wait.',
 'Wait.',
 'Wait.',
 'Wait.',
 'Wait.',
 'Begin.',
 'Begin.',
 'Do it.',
 'Hello!',
 'Hurry!',
 'I see.',
 'I see.',
 'I try.',
 'I won!',
 'Oh no!',
 'Relax.',
 'Shoot!',
 'Shoot!',
 'Smile.',
 'Attack!',
 'Cheers!',
 'Cheers!',
 'Eat it.',
 'Freeze!',
 'Freeze!',
 'Get up.',
 'Go now.',
 'Got it?',
 'Got it?',
 'He ran.',
 'Hop in.',
 'Hug me.',
 'I fell.',
 'I know.',
 'I left.',
 'I left.',
 'I left.',
 'I left.',
 'I left.',
 'I left.',
 'I lost.',
 'I quit.',
 "I'm OK.",
 'Listen.',
 'No way!',
 'Really?',
 'Thanks.',
 'Thanks.',
 'We try.',
 'Why me?',
 'Ask Tom.',
 'Ask Tom.',
 'Awesome!',
 'Be calm.',
 'Be cool.',
 'Be fair.',
 'Be kind.',
 'Be nice.',
 'Beat it.',
 'Call me.',
 'Call us.',
 'Come in.',
 'Come in.',
 'Come in.',
 '

In [101]:
pol_word

['Idź.',
 'Cześć.',
 'Uciekaj!',
 'Biegnij.',
 'Uciekaj.',
 'Kto?',
 'O, dziamdzia zaprzała jej szadź!',
 'Łał!',
 'Pali się!',
 'Strzelaj!',
 'Ognia!',
 'Pomocy!',
 'Skacz!',
 'Skok.',
 'Stój!',
 'Zatrzymaj się!',
 'Czekaj!',
 'Zaczekaj!',
 'Poczekaj!',
 'Czekajcie!',
 'Poczekajcie!',
 'Zaczekajcie!',
 'Niech pan zaczeka!',
 'Niech pani zaczeka!',
 'Czekajcie.',
 'Zaczekaj.',
 'Czekaj.',
 'Poczekaj.',
 'Poczekajcie.',
 'Zaczekajcie.',
 'Zaczynaj.',
 'Zaczynajcie.',
 'Zrób to.',
 'Cześć.',
 'Pośpiesz się!',
 'Rozumiem.',
 'Widzę.',
 'Próbuje.',
 'Wygrałem!',
 'O nie!',
 'Wyluzuj.',
 'Strzelaj!',
 'Ognia!',
 'Uśmiech.',
 'Atak!',
 'Na zdrowie!',
 'Twoje zdrowie!',
 'Zjedz to.',
 'Stój!',
 'Nie ruszaj się!',
 'Wstawaj.',
 'Idź już.',
 'Rozumiesz?',
 'Kapujesz?',
 'On pobiegł.',
 'Wskakuj.',
 'Przytul mnie.',
 'Przewróciłam się.',
 'Wiem.',
 'Wyjechałam.',
 'Odszedłem.',
 'Wyjechałam',
 'Wyszłam.',
 'Odeszłam.',
 'Wyszedłem.',
 'Przegrałem.',
 'Wychodzę.',
 'Ze mną wszystko w porządku.',


In [102]:
language_data = pd.DataFrame(columns=['English','Polish'])
language_data['English'] = english_word
language_data['Polish'] = pol_word

In [103]:
language_data

,English,Polish
0,Go.,Idź.
1,Hi.,Cześć.
2,Run!,Uciekaj!
3,Run.,Biegnij.
4,Run.,Uciekaj.
...,...,...
39995,Tom would've done that correctly the first tim...,"Tom zrobiłby to poprawnie za pierwszym razem, ..."
39996,"Tom, Mary, John and Alice were sitting around ...","Tom, Mary, John i Alice siedzieli dookoła stoł..."
39997,We consider it the citizens' legitimate right ...,Biblioteki publiczne uznajemy za podstawowe pr...
39998,We just want to ask him a few questions about ...,Chcemy tylko zadać mu kilka pytań na temat teg...


In [104]:
language_data.to_csv('language_data.csv', index=False)

In [105]:
english_text = language_data['English'].values
polish_text = language_data['Polish'].values
len(english_text), len(polish_text)

(40000, 40000)

## Data Cleaning

In [106]:
# converting all sentences to lower case
english_text = [x.lower() for x in english_text]
polish_text = [x.lower() for x in polish_text]

In [107]:
# removing inverted commas
english_text = [re.sub("'",'',x) for x in english_text]
polish_text = [re.sub("'",'',x) for x in polish_text]

In [108]:
english_text

['go.',
 'hi.',
 'run!',
 'run.',
 'run.',
 'who?',
 'wow!',
 'wow!',
 'fire!',
 'fire!',
 'fire!',
 'help!',
 'jump!',
 'jump.',
 'stop!',
 'stop!',
 'wait!',
 'wait!',
 'wait!',
 'wait!',
 'wait!',
 'wait!',
 'wait!',
 'wait!',
 'wait.',
 'wait.',
 'wait.',
 'wait.',
 'wait.',
 'wait.',
 'begin.',
 'begin.',
 'do it.',
 'hello!',
 'hurry!',
 'i see.',
 'i see.',
 'i try.',
 'i won!',
 'oh no!',
 'relax.',
 'shoot!',
 'shoot!',
 'smile.',
 'attack!',
 'cheers!',
 'cheers!',
 'eat it.',
 'freeze!',
 'freeze!',
 'get up.',
 'go now.',
 'got it?',
 'got it?',
 'he ran.',
 'hop in.',
 'hug me.',
 'i fell.',
 'i know.',
 'i left.',
 'i left.',
 'i left.',
 'i left.',
 'i left.',
 'i left.',
 'i lost.',
 'i quit.',
 'im ok.',
 'listen.',
 'no way!',
 'really?',
 'thanks.',
 'thanks.',
 'we try.',
 'why me?',
 'ask tom.',
 'ask tom.',
 'awesome!',
 'be calm.',
 'be cool.',
 'be fair.',
 'be kind.',
 'be nice.',
 'beat it.',
 'call me.',
 'call us.',
 'come in.',
 'come in.',
 'come in.',
 'c

### Removing Punctuations

In [109]:
def remove_punc(text_list):
    table = str.maketrans('', '', string.punctuation)
    removed_punc_text = []
    for sent in text_list:
        sentence = [word.translate(table) for word in sent.split(' ')]
        removed_punc_text.append(' '.join(sentence))
    return removed_punc_text

In [110]:
english_text = remove_punc(english_text)
polish_text = remove_punc(polish_text)


In [111]:
english_text

['go',
 'hi',
 'run',
 'run',
 'run',
 'who',
 'wow',
 'wow',
 'fire',
 'fire',
 'fire',
 'help',
 'jump',
 'jump',
 'stop',
 'stop',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'begin',
 'begin',
 'do it',
 'hello',
 'hurry',
 'i see',
 'i see',
 'i try',
 'i won',
 'oh no',
 'relax',
 'shoot',
 'shoot',
 'smile',
 'attack',
 'cheers',
 'cheers',
 'eat it',
 'freeze',
 'freeze',
 'get up',
 'go now',
 'got it',
 'got it',
 'he ran',
 'hop in',
 'hug me',
 'i fell',
 'i know',
 'i left',
 'i left',
 'i left',
 'i left',
 'i left',
 'i left',
 'i lost',
 'i quit',
 'im ok',
 'listen',
 'no way',
 'really',
 'thanks',
 'thanks',
 'we try',
 'why me',
 'ask tom',
 'ask tom',
 'awesome',
 'be calm',
 'be cool',
 'be fair',
 'be kind',
 'be nice',
 'beat it',
 'call me',
 'call us',
 'come in',
 'come in',
 'come in',
 'come in',
 'come in',
 'come in',
 'come in',
 'come in',
 'come in',
 'come on',
 'come o

In [112]:
polish_text

['idź',
 'cześć',
 'uciekaj',
 'biegnij',
 'uciekaj',
 'kto',
 'o dziamdzia zaprzała jej szadź',
 'łał',
 'pali się',
 'strzelaj',
 'ognia',
 'pomocy',
 'skacz',
 'skok',
 'stój',
 'zatrzymaj się',
 'czekaj',
 'zaczekaj',
 'poczekaj',
 'czekajcie',
 'poczekajcie',
 'zaczekajcie',
 'niech pan zaczeka',
 'niech pani zaczeka',
 'czekajcie',
 'zaczekaj',
 'czekaj',
 'poczekaj',
 'poczekajcie',
 'zaczekajcie',
 'zaczynaj',
 'zaczynajcie',
 'zrób to',
 'cześć',
 'pośpiesz się',
 'rozumiem',
 'widzę',
 'próbuje',
 'wygrałem',
 'o nie',
 'wyluzuj',
 'strzelaj',
 'ognia',
 'uśmiech',
 'atak',
 'na zdrowie',
 'twoje zdrowie',
 'zjedz to',
 'stój',
 'nie ruszaj się',
 'wstawaj',
 'idź już',
 'rozumiesz',
 'kapujesz',
 'on pobiegł',
 'wskakuj',
 'przytul mnie',
 'przewróciłam się',
 'wiem',
 'wyjechałam',
 'odszedłem',
 'wyjechałam',
 'wyszłam',
 'odeszłam',
 'wyszedłem',
 'przegrałem',
 'wychodzę',
 'ze mną wszystko w porządku',
 'słuchaj',
 'nie mam mowy',
 'naprawdę',
 'dziękuję',
 'dzięki',
 '

In [113]:
clear_digits = str.maketrans('', '', digits)
clear_digits_text = []

In [114]:
for sent in english_text:
    sentence = [word.translate(clear_digits) for word in sent.split(' ')]
    clear_digits_text.append(' '.join(sentence))

In [115]:
clear_digits_text

['go',
 'hi',
 'run',
 'run',
 'run',
 'who',
 'wow',
 'wow',
 'fire',
 'fire',
 'fire',
 'help',
 'jump',
 'jump',
 'stop',
 'stop',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'wait',
 'begin',
 'begin',
 'do it',
 'hello',
 'hurry',
 'i see',
 'i see',
 'i try',
 'i won',
 'oh no',
 'relax',
 'shoot',
 'shoot',
 'smile',
 'attack',
 'cheers',
 'cheers',
 'eat it',
 'freeze',
 'freeze',
 'get up',
 'go now',
 'got it',
 'got it',
 'he ran',
 'hop in',
 'hug me',
 'i fell',
 'i know',
 'i left',
 'i left',
 'i left',
 'i left',
 'i left',
 'i left',
 'i lost',
 'i quit',
 'im ok',
 'listen',
 'no way',
 'really',
 'thanks',
 'thanks',
 'we try',
 'why me',
 'ask tom',
 'ask tom',
 'awesome',
 'be calm',
 'be cool',
 'be fair',
 'be kind',
 'be nice',
 'beat it',
 'call me',
 'call us',
 'come in',
 'come in',
 'come in',
 'come in',
 'come in',
 'come in',
 'come in',
 'come in',
 'come in',
 'come on',
 'come o

In [116]:
polish_text = [re.sub("[1234567890]","",x) for x in polish_text]

In [117]:
polish_text = [re.sub("[\u200d]","",x) for x in polish_text]

In [118]:
# triming the sentences in both the texts
english_text = [x.strip() for x in english_text]
polish_text = [x.strip() for x in polish_text]

In [119]:
# adding start and end tag to polish sentences so the decoder understands when to start and when to stop
polish_text = ["start " + x + " end" for x in polish_text]

In [120]:
polish_text[0]

'start idź end'

In [121]:
english_text[0],polish_text[0]

('go', 'start idź end')

In [122]:
X = english_text
y = polish_text

In [123]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.1,train_size=0.9,random_state=0)

In [124]:
# determining the maximum length of sentence for pad sequences for english
max_english_text = max([len(x.split(' ')) for x in X_train])
max_english_text

17

In [125]:
# determining the maximum length of sentence for pad sequences for polish
max_polish_text = max([len(x.split(' ')) for x in y_train])
max_polish_text

20

In [126]:
# similarly we find it for the test set as well
max_test_english_text = max([len(x.split(' ')) for x in X_test])
max_test_english_text

17

In [127]:
max_test_polish_text = max([len(x.split(' ')) for x in y_test])
max_test_polish_text

16

### Tokenizing the data

In [128]:
# english text
tokenize_eng = Tokenizer()
tokenize_eng.fit_on_texts(X_train)

In [129]:
word2index_eng = tokenize_eng.word_index

In [130]:
word2index_eng

{'i': 1,
 'to': 2,
 'tom': 3,
 'the': 4,
 'you': 5,
 'a': 6,
 'is': 7,
 'that': 8,
 'he': 9,
 'in': 10,
 'it': 11,
 'do': 12,
 'have': 13,
 'was': 14,
 'me': 15,
 'of': 16,
 'this': 17,
 'my': 18,
 'dont': 19,
 'we': 20,
 'for': 21,
 'what': 22,
 'are': 23,
 'im': 24,
 'be': 25,
 'your': 26,
 'know': 27,
 'mary': 28,
 'his': 29,
 'like': 30,
 'not': 31,
 'she': 32,
 'want': 33,
 'on': 34,
 'with': 35,
 'and': 36,
 'at': 37,
 'can': 38,
 'how': 39,
 'go': 40,
 'didnt': 41,
 'were': 42,
 'its': 43,
 'about': 44,
 'did': 45,
 'here': 46,
 'has': 47,
 'think': 48,
 'youre': 49,
 'will': 50,
 'there': 51,
 'all': 52,
 'cant': 53,
 'him': 54,
 'going': 55,
 'time': 56,
 'as': 57,
 'they': 58,
 'very': 59,
 'why': 60,
 'up': 61,
 'had': 62,
 'get': 63,
 'her': 64,
 'out': 65,
 'if': 66,
 'just': 67,
 'ill': 68,
 'who': 69,
 'an': 70,
 'would': 71,
 'ive': 72,
 'from': 73,
 'no': 74,
 'been': 75,
 'need': 76,
 'tell': 77,
 'help': 78,
 'could': 79,
 'one': 80,
 'when': 81,
 'come': 82,
 'never

In [131]:
vocab_size_eng = len(word2index_eng) + 1
vocab_size_eng

7921

In [133]:
X_train

['at any rate the program was a success',
 'tom died at the age of 97',
 'tom has been experiencing severe headaches and dizziness',
 'lets talk about what happened in boston',
 'we didnt kill anyone',
 'he knows how to sell houses',
 'i avoid tom',
 'has anyone seen my key',
 'does tom know that you like mary',
 'tom and i had a fight',
 'tom bought the cheapest thing in the store',
 'try again',
 'forget it',
 'that matter was decided by the supreme court',
 'i know several canadians',
 'i locked the door',
 'tom looked really busy',
 'is tom your husband',
 'i wouldnt think of it',
 'the noise is getting louder and louder',
 'you never told me that',
 'tom understands what mary wants him to do',
 'he solved the difficult problem',
 'i want more detailed information',
 'do you have any peppermint candy',
 'i saw a dog',
 'i flushed the toilet',
 'tom hasnt ever been to boston',
 'it was an incredible sight',
 'hes inclined to violence',
 'i have the same dictionary as your brother ha

In [134]:
X_train = tokenize_eng.texts_to_sequences(X_train)
X_train

[[37, 111, 1530, 4, 1644, 14, 6, 881],
 [3, 323, 37, 4, 574, 16, 4838],
 [3, 47, 75, 4839, 4840, 3659, 36, 4841],
 [120, 140, 44, 22, 185, 10, 84],
 [20, 41, 545, 199],
 [9, 213, 39, 2, 864, 1754],
 [1, 1302, 3],
 [47, 199, 247, 18, 595],
 [126, 3, 27, 8, 5, 30, 28],
 [3, 36, 1, 62, 6, 651],
 [3, 237, 4, 3660, 272, 10, 4, 665],
 [204, 141],
 [372, 11],
 [8, 421, 14, 334, 87, 4, 4842, 3661],
 [1, 27, 575, 2294],
 [1, 844, 4, 219],
 [3, 310, 97, 206],
 [7, 3, 26, 700],
 [1, 244, 48, 16, 11],
 [4, 652, 7, 302, 2295, 36, 2295],
 [5, 83, 98, 15, 8],
 [3, 1755, 22, 28, 203, 54, 2, 12],
 [9, 1192, 4, 389, 192],
 [1, 33, 108, 2591, 1004],
 [12, 5, 13, 111, 4843, 1645],
 [1, 223, 6, 201],
 [1, 3662, 4, 1373],
 [3, 491, 178, 75, 2, 84],
 [11, 14, 70, 1646, 1374],
 [153, 4844, 2, 2296],
 [1, 13, 4, 233, 532, 57, 26, 367, 47],
 [74, 80, 47, 75, 46],
 [4, 943, 7, 2071, 36, 11, 50, 353],
 [81, 45, 5, 77, 15],
 [72, 733, 104],
 [43, 67, 6, 606],
 [1, 27, 26, 180],
 [4, 596, 7, 1193, 10, 4, 175],
 [32

In [135]:
X_train = pad_sequences(X_train, maxlen=max_english_text, padding='post')

In [136]:
X_test = tokenize_eng.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen = max_english_text, padding='post')

In [137]:
# tokenizing polish text
tokenize_pol = Tokenizer()
tokenize_pol.fit_on_texts(y_train)
word2index_pol = tokenize_pol.word_index
vocab_size_pol = len(word2index_pol) + 1
vocab_size_pol

19516

In [138]:
y_train = tokenize_pol.texts_to_sequences(y_train)
y_train = pad_sequences(y_train, maxlen=max_polish_text, padding='post')
y_test = tokenize_pol.texts_to_sequences(y_test)
y_test = pad_sequences(y_test, maxlen = max_polish_text, padding='post')

In [139]:
X_train[0]

array([  37,  111, 1530,    4, 1644,   14,    6,  881,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [140]:
y_train[0]

array([   1,    8,  966, 1182, 1567,    5, 6164,    2,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [141]:
with open('NMT_data.pkl','wb') as f:
  pkl.dump([X_train, y_train, X_test, y_test],f)
with open('NMT_Etokenizer.pkl','wb') as f:
  pkl.dump([vocab_size_eng, word2index_eng, tokenize_eng], f)
with open('NMT_Ptokenizer.pkl', 'wb') as f:
  pkl.dump([vocab_size_pol, word2index_pol, tokenize_pol], f)

In [142]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [46]:
import torch

In [47]:
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [144]:
# def max_length(lines):
# 	return max(len(line.split()) for line in lines)
# eng_timesteps = max_length()
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

In [154]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((36000, 17), (36000, 20), (4000, 17), (4000, 20))

In [151]:
n_units=256
model = Sequential()
# model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
model.add(Embedding(vocab_size_eng,256,input_length=max_test_english_text, mask_zero=True))
model.add(LSTM(n_units))
model.add(RepeatVector(max_test_polish_text))
model.add(LSTM(n_units, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size_pol, activation='softmax')))
model.compile(optimizer='adam', loss='categorical_crossentropy')
print(model.summary())
# plot_model(model, to_file='model.png', show_shapes=True)
filename = 'models.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(X_train, y_train, epochs=30, batch_size=64, validation_data=(X_test, y_test), callbacks=[checkpoint], verbose=2)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 17, 256)           2027776   
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 16, 256)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 16, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 16, 19516)         5015612   
Total params: 8,094,012
Trainable params: 8,094,012
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30


ValueError: in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:533 train_step  **
        y, y_pred, sample_weight, regularization_losses=self.losses)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\compile_utils.py:205 __call__
